# i-TED E characterization
## Based on v5 from Multi i_TED

In [1]:
pkg_ver = lambda pkg: "{:<20}{:}".format(pkg.__name__,pkg.__version__)

# ROOT
import uproot
print(pkg_ver(uproot))
import ROOT

# Machine Learning
import sklearn
print(pkg_ver(sklearn))
import torch
print(pkg_ver(torch))

# Data science
import scipy
print(pkg_ver(scipy))
import numpy
print(pkg_ver(numpy))
import pandas
print(pkg_ver(pandas))

# Visualizations
import matplotlib
print(pkg_ver(matplotlib))
import matplotlib.pyplot as plt

import tqdm
print(pkg_ver(tqdm))

import glob

uproot              4.3.5
Welcome to JupyROOT 6.28/04
sklearn             1.2.2
torch               2.0.1
scipy               1.10.1
numpy               1.24.3
pandas              1.5.3
matplotlib          3.7.1
tqdm                4.65.0


In [2]:
%jsroot

In [3]:
class spectrum:
    
    def __init__(self, File_, iTED_, Crystal_, Configuration_, Window_, Calibration_, Run_):  

        self.__File = ROOT.TFile.Open(File_,"READ")
        self.__iTED = iTED_
        self.__Crystal = Crystal_
        self.__Configuration = Configuration_
        self.__Window = Window_
        self.__Calibration = Calibration_
        self.__Run = Run_
        
    def __call__(self, ch):
        return numpy.polyval(self.__Calibration[::-1],ch)
    
    def __ch__(self, en):
        p = numpy.poly1d(self.__Calibration[::-1])
        temp  = set(i for i in (p - en).roots if i > 0)
        return list(temp).pop()

    def File(self):
        return self.__File
    
    def TH1D(self):
        if self.__iTED == "E":
            match self.__Crystal:
                case 0:
                    i = "A"
                    c = 0
                case 1:
                    i = "A"
                    c = 2
                case 2:
                    i = "A"
                    c = 3
                case 3:
                    i = "A"
                    c = 4
                case 4:
                    i = "B"
                    c = 0
        else:
            i = self.__iTED
            c = self.__Crystal

        #print(self.__iTED,self.__Crystal,"->",i,c)
            
        self.__TH1D = self.__File.Get(
                            "{}_{}_amplitude_spectra;1".format(
                                "SCATTERER" if c==0 else "ABSORBER",
                                i if c==0 else "{}_{}".format(i,c)
                            )
                        )
        
        return self.__TH1D
    
    def Calibration(self):
        return self.__Calibration
    
    def iTED(self):
        return self.__iTED
    
    def Crystal(self):
        return self.__Crystal
    
    def Configuration(self):
        return self.__Configuration
    
    def Window(self):
        return self.__Window
    
    def Rate(self, Time_):
        return self.__TH1D.Integral()/Time_
    
    def Alpha(self, Time_):
        return self.__TH1D.Integral(
            self.__TH1D.FindBin(self.__ch__(1600)),
            self.__TH1D.FindBin(self.__ch__(2800))
        )/Time_
    
    def __repr__(self):
        return "iTED: {},Crystal: {},Configuration: {},Window: {}, Run: {}".format(
            self.__iTED,
            self.__Crystal,
            self.__Configuration,
            self.__Window,
            self.__Run
        )
    
    def __str__(self):
        return "{}.{}.{}.{}.{}".format(
            self.__iTED,
            self.__Crystal,
            self.__Configuration,
            self.__Window,
            self.__Run
        )

In [4]:
calp = glob.glob('/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/Calibration/**/*.CALp', recursive=True)

calibrations = []

for i in calp:
    line = pandas.read_csv(
        i,
        sep = "      :  ",
        skiprows=[0,4,5,6],
        header=None,
        engine="python"
    ).drop([0], axis=1).T
    
    line["crystal"] = i.split("/")[-1][:2]
    line["cw"] = i.split("/")[-1][5:8]
    
    calibrations.append(line)
    
calibrations_df = pandas.concat(calibrations)

In [5]:
iTEDE = {
    "88c_1":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_31/Cs137_D.2023_05_31_T.18_13_48_C.itedE_2023.05.30_4.0v_887C_15s_serie1_25_CW{win}.root",
    "88c_2":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_31/Cs137_D.2023_05_31_T.18_14_05_C.itedE_2023.05.30_4.0v_887C_15s_serie2_25_CW{win}.root",
    "88c_3":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_31/Cs137_D.2023_05_31_T.18_14_22_C.itedE_2023.05.30_4.0v_887C_15s_serie3_25_CW{win}.root",
    "88c_4":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_31/Cs137_D.2023_05_31_T.18_14_39_C.itedE_2023.05.30_4.0v_887C_15s_serie4_25_CW{win}.root",
    "88c_5":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_31/Cs137_D.2023_05_31_T.18_14_55_C.itedE_2023.05.30_4.0v_887C_15s_serie5_25_CW{win}.root",
    "88c_6":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_31/Cs137_D.2023_05_31_T.18_15_12_C.itedE_2023.05.30_4.0v_887C_15s_serie6_25_CW{win}.root",
    "88c_7":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_31/Cs137_D.2023_05_31_T.18_15_29_C.itedE_2023.05.30_4.0v_887C_15s_serie7_25_CW{win}.root",
    "88c_8":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_31/Cs137_D.2023_05_31_T.18_15_46_C.itedE_2023.05.30_4.0v_887C_15s_serie8_25_CW{win}.root",
    "88c_9":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_31/Cs137_D.2023_05_31_T.18_16_03_C.itedE_2023.05.30_4.0v_887C_15s_serie9_25_CW{win}.root",
    "88c_10": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_31/Cs137_D.2023_05_31_T.18_16_20_C.itedE_2023.05.30_4.0v_887C_15s_serie10_25_CW{win}.root",
    "88c_11": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_31/Cs137_D.2023_05_31_T.18_16_37_C.itedE_2023.05.30_4.0v_887C_15s_serie11_25_CW{win}.root",
    "88c_12": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_31/Cs137_D.2023_05_31_T.18_16_54_C.itedE_2023.05.30_4.0v_887C_15s_serie12_25_CW{win}.root",
    "88c_13": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_31/Cs137_D.2023_05_31_T.18_17_11_C.itedE_2023.05.30_4.0v_887C_15s_serie13_25_CW{win}.root",
    "88c_14": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_31/Cs137_D.2023_05_31_T.18_17_28_C.itedE_2023.05.30_4.0v_887C_15s_serie14_25_CW{win}.root",
    "88c_15": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_31/Cs137_D.2023_05_31_T.18_17_45_C.itedE_2023.05.30_4.0v_887C_15s_serie15_25_CW{win}.root",
    "88c_16": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_31/Cs137_D.2023_05_31_T.18_18_02_C.itedE_2023.05.30_4.0v_887C_15s_serie16_25_CW{win}.root",
    "88c_17": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_31/Cs137_D.2023_05_31_T.18_18_19_C.itedE_2023.05.30_4.0v_887C_15s_serie17_25_CW{win}.root",
    "88c_18": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_31/Cs137_D.2023_05_31_T.18_18_36_C.itedE_2023.05.30_4.0v_887C_15s_serie18_25_CW{win}.root",
    "88c_19": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_31/Cs137_D.2023_05_31_T.18_18_53_C.itedE_2023.05.30_4.0v_887C_15s_serie19_25_CW{win}.root",
    "88c_20": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_31/Cs137_D.2023_05_31_T.18_19_10_C.itedE_2023.05.30_4.0v_887C_15s_serie20_25_CW{win}.root",
    "88c_21": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_31/Cs137_D.2023_05_31_T.18_19_27_C.itedE_2023.05.30_4.0v_887C_15s_serie21_25_CW{win}.root",
    "88c_22": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_31/Cs137_D.2023_05_31_T.18_19_44_C.itedE_2023.05.30_4.0v_887C_15s_serie22_25_CW{win}.root",
    "88c_23": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_31/Cs137_D.2023_05_31_T.18_20_01_C.itedE_2023.05.30_4.0v_887C_15s_serie23_25_CW{win}.root",
    "88c_24": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_31/Cs137_D.2023_05_31_T.18_20_18_C.itedE_2023.05.30_4.0v_887C_15s_serie24_25_CW{win}.root",
    "88c_25": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_31/Cs137_D.2023_05_31_T.18_20_35_C.itedE_2023.05.30_4.0v_887C_15s_serie25_25_CW{win}.root",
}

In [6]:
def get_resolution(cell):
    
    TH1D = cell.TH1D()
    
    TH1D.GetXaxis().SetRange(TH1D.FindBin(100),TH1D.FindBin(400))
    
    MaxBin   = TH1D.FindBin(TH1D.GetMaximumBin())
    
    ADC_Low  = MaxBin-50
    ADC_High = MaxBin+40

    p1 = 60 if (cell.iTED()=="E" and cell.Crystal()==4) else 1000
    p2 = -0.2 if (cell.iTED()=="E" and cell.Crystal()==4) else -1

    gaussFit = ROOT.TF1("gaussFit", "gaus(0)+pol1(3)", ADC_Low, ADC_High)
    gaussFit.SetParameters(TH1D.GetMaximum(),MaxBin,7,p1,p2,0)
    TH1D.Fit(gaussFit,"QR")
    
    sigma = abs(gaussFit.GetParameter(2))
    centroid_ch = gaussFit.GetParameter(1)
    
    x1 = cell(centroid_ch+sigma*numpy.sqrt(2*numpy.log(2)))
    x2 = cell(centroid_ch-sigma*numpy.sqrt(2*numpy.log(2)))
    centroid = cell(centroid_ch)
        
    fwhm = x1-x2
            
    return sigma*numpy.sqrt(2*numpy.log(2))*2/centroid_ch*100, (fwhm/centroid)*100, centroid

In [7]:
def TH1D_draw(cell):
    
    TH1D = cell.TH1D()
    
    canvas = ROOT.TCanvas()
    canvas.cd()
    
    TH1D.SetTitle(repr(cell))
    TH1D.SetStats(False)
    
    latex = ROOT.TLatex()
    latex.SetNDC()
    latex.SetTextSize(0.03)
    
    TH1D.Draw("pe")
    
    l1,l2,l3 = get_resolution(cell)

    ited = cell.iTED()
    
    #l4 = uproot.open(f"../../data/nTOF_March2022/888/CW100ns/Resolutions_Cs137_CenterScatter_iTED{ited}_8.8.8_100ns.root:grResolEnergy;1").values()[1][cell.Crystal()]
    
    latex.DrawText(0.7, 0.8, "R_ch: {:.2f}%".format(l1))
    
    latex.DrawText(0.7, 0.75, "R_E: {:.2f}%".format(l2))
        
    if 662*(1-l2/100) < l3 < 662*(1+l2/100):
        latex.DrawText(0.7, 0.7, "E: {:.0f}keV".format(l3))
    else:
        latex.DrawText(0.7, 0.7, "->E: {:.0f}keV".format(l3))
            
    return canvas

In [8]:
entries = []

for iTED in [4]:
    for Crystal in tqdm.tqdm([0,1,2,3,4]):
        for Configuration in ["88c"]:
            for CW in tqdm.tqdm([100,200]):
                for i_run in range(1,25+1):
                    
                    cryst_code = ["A", "B", "C", "D", "E"][iTED]+str(Crystal)
                                        
                    iTEDE[f"{Configuration}_{i_run}_{CW}"] = iTEDE[f"{Configuration}_{i_run}"](CW)
                                                    
                    spectr = spectrum(
                        iTEDE[f"{Configuration}_{i_run}"](CW),
                        ["A","B","C","D","E"][iTED], 
                        Crystal,
                        Configuration, 
                        CW, 
                        calibrations_df.query("crystal == @cryst_code & cw == '100'").iloc[0][[0,1,2]],
                        i_run
                    )
                                    
                    entries.append(
                        pandas.DataFrame({
                            "resolution": get_resolution(spectr)[1],
                            "fit": abs(get_resolution(spectr)[2]-662)/662*100,
                            "cps": spectr.Rate(30),
                            "alpha": spectr.Alpha(30),
                            "iTED": ["A", "B", "C", "D", "E"][iTED],
                            "crystal": Crystal,
                            "cw": CW,
                            "configuration": Configuration,
                            "run": i_run,
                            "obj": spectr,
                        }, index=[0])
                    )
                    
entries_df = pandas.concat(entries, ignore_index=True)

100%|██████████████████| 5/5 [00:29<00:00,  5.99s/it]
Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1


## Analysis

### Resolution

#### per iTED per crystal per configuration per window

In [9]:
entries_df.groupby(["iTED","cw","configuration"]).resolution.describe().drop(['count', 'std'], axis=1).T.style.background_gradient(cmap ='YlOrRd',axis=1)

In [10]:
print("The distribution of the resolution standard deviation in the runs:")
entries_df.groupby(["iTED","crystal","cw","configuration"]).resolution.describe().drop(['count', '25%', '50%', '75%'], axis=1)["std"].describe().drop(['count', 'mean', 'std'], axis=0)

The distribution of the resolution standard deviation in the runs:


min    0.074474
25%    0.082948
50%    0.092500
75%    0.095989
max    0.162634
Name: std, dtype: float64

#### per crystal

In [11]:
entries_df.groupby(["crystal"]).resolution.describe().drop(['count', 'std'], axis=1).T.style.background_gradient(cmap ='YlOrRd',axis=None)

crystal,0,1,2,3,4
mean,5.788094,8.409695,6.989460,10.551853,10.473712
min,5.556656,8.282983,6.734825,10.402813,10.200300
25%,5.731218,8.349889,6.869465,10.498198,10.350295
50%,5.801771,8.409264,6.957128,10.574571,10.484832
75%,5.840898,8.450800,7.084359,10.613587,10.566598
max,5.965100,8.572792,7.376909,10.706116,10.888355


#### per window

In [12]:
entries_df.groupby(["cw"]).resolution.describe().drop(['count', 'std'], axis=1).T.style.background_gradient(cmap ='YlOrRd',axis=1)

cw,100,200
mean,8.462887,8.422238
min,5.562662,5.556656
25%,6.869456,6.869493
50%,8.409369,8.409159
75%,10.522398,10.401446
max,10.888355,10.706116


#### per iTED

In [13]:
entries_df.groupby(["iTED"]).resolution.describe().drop(['count', 'std'], axis=1).T.style.background_gradient(cmap ='YlOrRd',axis=None)

iTED,E
mean,8.442563
min,5.556656
25%,6.869465
50%,8.409264
75%,10.488420
max,10.888355


#### per parameters (conf and window)

In [14]:
entries_df.groupby(["configuration","cw"]).resolution.describe().drop(['count', 'std'], axis=1).T.style.background_gradient(cmap ='YlOrRd',axis=None)

### Fit

#### per parameters

In [15]:
entries_df.groupby(["configuration","cw"]).fit.describe().drop(['count', 'std'], axis=1).T.style.background_gradient(cmap ='YlOrRd',axis=None)

#### per run

In [16]:
entries_df.groupby(["iTED","crystal","configuration"]).fit.describe().drop(['count', 'std'], axis=1).T.style.background_gradient(cmap ='YlOrRd',axis=None)

### Count rate using integral

In [17]:
entries_df.groupby(["iTED","crystal"]).cps.describe().drop(['count', 'std'], axis=1).T.style.background_gradient(cmap ='YlOrRd',axis=1)

### Alpha activity rate using 1600keV < E < 2800keV

In [18]:
entries_df.groupby(["iTED","crystal"]).alpha.describe().drop(['count', 'std'], axis=1).T.style.background_gradient(cmap ='YlOrRd',axis=1)

## Draw fits

In [19]:
entries_df.head()

,resolution,fit,cps,alpha,iTED,crystal,cw,configuration,run,obj
0,5.841176,2.481385,789.200000,55.766667,E,0,100,88c,1,E.0.88c.100.1
1,5.679592,1.818201,802.133333,54.266667,E,0,100,88c,2,E.0.88c.100.2
2,5.856268,2.010987,798.266667,56.233333,E,0,100,88c,3,E.0.88c.100.3
3,5.750958,2.037359,792.833333,56.566667,E,0,100,88c,4,E.0.88c.100.4
4,5.965100,1.945965,791.566667,56.366667,E,0,100,88c,5,E.0.88c.100.5


In [22]:
for index, row in entries_df.query("run==4").iterrows():
    TH1D_draw(row.obj).Draw()